In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import duckdb
import polars as pl
import sys
sys.path.append('../') 
import upsert_utils

In [3]:
duckdb.sql("""
attach '../../../db.sqlite3';
use db;
""")

In [4]:
duckdb.sql("""
begin transaction;
""")

In [86]:
duckdb.sql("""
rollback;
""")

TransactionException: TransactionContext Error: cannot rollback - no transaction is active

In [5]:
all_columns = duckdb.sql("""
select row_number() over () as csv_row_id, * from read_csv('ProdModule.csv');
""").pl()

In [6]:
ob_dimensions = duckdb.sql(r"""
select
    csv_row_id,
    columns('^Dimension__(.+)') as '\1',
from all_columns
""")
ob_dimensions

┌────────────┬────────────┬─────────────┬─────────────┬──────────────┐
│ csv_row_id │ Width_Unit │ Width_Value │ Height_Unit │ Height_Value │
│   int64    │  varchar   │   double    │   varchar   │    double    │
├────────────┼────────────┼─────────────┼─────────────┼──────────────┤
│          1 │ m          │       0.992 │ m           │         1.64 │
│          2 │ m          │       0.992 │ m           │         1.64 │
│          3 │ m          │       0.992 │ m           │         1.64 │
│          4 │ m          │       0.992 │ m           │         1.64 │
│          5 │ m          │       0.992 │ m           │         1.64 │
│          6 │ m          │       0.995 │ m           │        1.646 │
│          7 │ m          │       0.995 │ m           │        1.646 │
│          8 │ m          │       0.995 │ m           │        1.646 │
│          9 │ m          │       0.995 │ m           │        1.646 │
│         10 │ m          │       0.995 │ m           │        1.646 │
│     

In [7]:
ob_prod_cells = duckdb.sql(r"""
select
    csv_row_id,
    columns('^ProdCell__(.+)') as '\1',
from all_columns
""")
ob_prod_cells

┌────────────┬──────────────────────────┐
│ csv_row_id │ CellTechnologyType_Value │
│   int64    │         varchar          │
├────────────┼──────────────────────────┤
│          1 │ MonoSi                   │
│          2 │ MonoSi                   │
│          3 │ MonoSi                   │
│          4 │ MonoSi                   │
│          5 │ MonoSi                   │
│          6 │ PolySi                   │
│          7 │ PolySi                   │
│          8 │ PolySi                   │
│          9 │ MonoSi                   │
│         10 │ PolySi                   │
│          · │   ·                      │
│          · │   ·                      │
│          · │   ·                      │
│       9991 │ MonoSi                   │
│       9992 │ MonoSi                   │
│       9993 │ MonoSi                   │
│       9994 │ MonoSi                   │
│       9995 │ MonoSi                   │
│       9996 │ MonoSi                   │
│       9997 │ MonoSi             

In [8]:
ob_prod_modules = all_columns.drop('^.+__.+$')
duckdb.sql("""
describe ob_prod_modules;
""")

┌─────────────────────────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│                   column_name                   │ column_type │  null   │   key   │ default │  extra  │
│                     varchar                     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ csv_row_id                                      │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ id                                              │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ ProdMfr_Value                                   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ProdCode_Value                                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Description_Value                               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ModuleArea_Unit                             

In [11]:
ob_prodmodule_ids = upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select
        csv_row_id,
        ModuleArea_Unit,
        ModuleArea_Value,
        CellsInSeries_Value,
        CellStringsParallelQuantity_Value,
        IsBIPV_Value,
        TemperatureNOCT_Unit,
        TemperatureNOCT_Value,
        TemperatureCoefficientMaximumPower_Unit,
        TemperatureCoefficientMaximumPower_Value,
        TemperatureCoefficientShortCircuitCurrent_Unit,
        TemperatureCoefficientShortCircuitCurrent_Value,
        TemperatureCoefficientOpenCircuitVoltage_Unit,
        TemperatureCoefficientOpenCircuitVoltage_Value,
        TemperatureCoefficientMaxPowerCurrent_Unit,
        TemperatureCoefficientMaxPowerCurrent_Value,
        TemperatureCoefficientMaxPowerVoltage_Unit,
        TemperatureCoefficientMaxPowerVoltage_Value,
    from ob_prod_modules
    """),
    target_table_name='server_prodmodule',
    pk='product_ptr_id',
    fk_fills=[
        ('product_ptr_id', dict(
            rows=duckdb.sql("""
            select
                csv_row_id,
                replace(cast(uuidv4() as varchar), '-', '') as ProdID_Value,
                ProdMfr_Value,
                ProdCode_Value,
                coalesce(Description_Value, '') as Description_Value,
                'ProdModule' as ProdType_Value,
            from ob_prod_modules
            """),
            target_table_name='server_product',
            pk='id',
            superclass=True,
            fk_fills=[
                ('Dimension_id', dict(target_table_name='server_dimension', pk='id')),
            ]
        )),
        ('ProdCell_id', dict(
            rows=ob_prod_cells,
            target_table_name='server_prodcell',
            pk='product_ptr_id',
            fk_fills=[
                ('product_ptr_id', dict(
                    rows=duckdb.sql("""
                    select
                        csv_row_id,
                        replace(cast(uuidv4() as varchar), '-', '') as ProdID_Value,
                        null as ProdCode_Value,
                        'ProdCell' as ProdType_Value,
                    from ob_prod_cells
                    """),
                    target_table_name='server_product',
                    pk='id',
                    superclass=True,
                    fk_fills=[
                        ('Dimension_id', dict(target_table_name='server_dimension', pk='id')),
                    ]
                )),
            ]
        )),
        ('ProdGlazing_id', dict(target_table_name='server_prodglazing', pk='id')),
    ]
)
ob_prodmodule_ids

┌────────────┬────────────────┐
│ csv_row_id │ product_ptr_id │
│   int64    │     int64      │
├────────────┼────────────────┤
│          1 │              1 │
│          2 │              2 │
│          3 │              3 │
│          4 │              4 │
│          5 │              5 │
│          6 │              6 │
│          7 │              7 │
│          8 │              8 │
│          9 │              9 │
│         10 │             10 │
│          · │              · │
│          · │              · │
│          · │              · │
│       9991 │           9991 │
│       9992 │           9992 │
│       9993 │           9993 │
│       9994 │           9994 │
│       9995 │           9995 │
│       9996 │           9996 │
│       9997 │           9997 │
│       9998 │           9998 │
│       9999 │           9999 │
│      10000 │          10000 │
├────────────┴────────────────┤
│ ? rows            2 columns │
│ (>9999 rows, 20 shown)      │
└─────────────────────────────┘

In [12]:
duckdb.sql("""
commit;
""")